In [52]:
import numpy as np
import pandas as pd
import itertools
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import pyplot
from scipy import stats
import os
import time
from sklearn import preprocessing, feature_extraction, model_selection
from copy import deepcopy
from IPython.display import display, HTML
import scipy.sparse
from scipy.spatial.distance import cdist
from scipy.spatial import KDTree
import networkx as nx

In [53]:
!ls

20210413_ContMRTX_neighbour_clustering_61clusters_DCsSeparated_TCrenamed_addedInfo copy.csv
Figure_1.png
__pycache__
data
imc_data_preprocess.ipynb
layer.py
main.py
plots
stolen.py
utils.py
utils2.py


In [54]:
full2 = pd.read_csv('20210413_ContMRTX_neighbour_clustering_61clusters_DCsSeparated_TCrenamed_addedInfo copy.csv')
full2_nandrop=full2.dropna()

In [55]:
full2['Filename'].unique(), full2['ROI_name'].unique()

(array(['20190913_BRAC3529.2d_ROI1_MRTX',
        '20200130_BRAC4002.3c_ROI2_MRTX_crop1',
        '20200130_BRAC4002.3c_ROI2_MRTX_crop2',
        '20200130_BRAC4002.3c_ROI3_MRTX',
        '20190917_BRAC3495.3f_ROI1_Cont_crop1',
        '20190917_BRAC3495.3f_ROI1_Cont_crop2',
        '20190927_BRAC3529.2b_ROI1_MRTX_crop2',
        '20191119_BRAC3326.4e_ROI1_Cont_crop1',
        '20191121_BRAC3438.6f_ROI1_Cont', '20191121_BRAC3438.6f_ROI2_Cont',
        '20191121_BRAC3438.6f_ROI3_Cont', '20200130_BRAC4002.3c_ROI1_MRTX'],
       dtype=object),
 array(['BRAC3529.2d_ROI1_MRTX.', 'BRAC4002.3c_ROI2_MRTX_crop1.',
        'BRAC4002.3c_ROI2_MRTX_crop2.', 'BRAC4002.3c_ROI3_MRTX.',
        'BRAC3495.3f_ROI1_Cont_crop1.', 'BRAC3495.3f_ROI1_Cont_crop2.',
        'BRAC3529.2b_ROI1_MRTX_crop2.', 'BRAC3326.4e_ROI1_Cont_crop1.',
        'BRAC3438.6f_ROI1_Cont.', 'BRAC3438.6f_ROI2_Cont.',
        'BRAC3438.6f_ROI3_Cont.', 'BRAC4002.3c_ROI1_MRTX.'], dtype=object))

In [56]:
full2.columns

Index(['Unnamed: 0', 'X.3', 'X.2', 'X.1', 'X', 'cellID', 'cellType', 'B_cells',
       'Epithelium', 'Fibroblasts', 'Myeloid_dendritic_cells',
       'Myeloid_dendritic_cells_CD103', 'Myeloid_macrophages',
       'Myeloid_macrophages_type1', 'Myeloid_macrophages_type2',
       'Myeloid_neutrophils', 'Normal', 'Tumour', 'Vessels', 'T_cells_CD8',
       'T_cells_CD4', 'T_reg_cells', 'NK_cells', 'cluster', 'Treatment',
       'Domain', 'ROI_name', 'Filename', 'Clustername', 'ObjectNumber',
       'ImageNumber', 'Location_Center_X', 'Location_Center_Y', 'MI_B220',
       'MI_CD103', 'MI_CD11c', 'MI_CD206', 'MI_CD3', 'MI_CD44', 'MI_CD45',
       'MI_CD4', 'MI_CD68', 'MI_CD80', 'MI_CD86', 'MI_CD8', 'MI_EPCAM',
       'MI_F480', 'MI_Foxp3', 'MI_Ki67', 'MI_LY6G', 'MI_MHCcII', 'MI_NKp46',
       'MI_PD1', 'MI_PDL1', 'MI_PECAM', 'MI_PVR', 'MI_TCRgd', 'MI_Vimentin',
       'MI_aSMA', 'MI_casp3', 'MI_pS6', 'clustername_tumour',
       'clustername_tumourID'],
      dtype='object')

In [57]:
# Number of treated (MRTX) and control (Control) cells
print(len(full2_nandrop))
t=full2_nandrop['cluster'].value_counts().to_frame()
valid_clusters=t.nlargest(20,'cluster').index
full2_nandrop=full2_nandrop[full2_nandrop['cluster'].isin( valid_clusters)]
print(len(full2_nandrop))
t.nlargest(20,'cluster')

276759
230394


,cluster
18,33915
46,27145
60,22347
39,18722
40,15983
58,13377
34,11598
44,11494
61,10845
16,10625


In [6]:
full2_nandrop['Treatment'].value_counts().to_frame()

,Treatment
MRTX,144898
Control,131861


In [7]:
# MRTX and Control data frames
df_mrtx = full2_nandrop[full2_nandrop['Treatment'] == 'MRTX']
df_control = full2_nandrop[full2_nandrop['Treatment'] == 'Control']

In [8]:
# Which columns have data type 'object' entries? Some of these will need to be label encoded, or one-hot encoded
df_control.select_dtypes(include='object')

,cellID,cellType,Treatment,Domain,ROI_name,Filename,Clustername,clustername_tumour
104818,2_1,Vessels,Control,normal,BRAC3495.3f_ROI1_Cont_crop1.,20190917_BRAC3495.3f_ROI1_Cont_crop1,"14_Endothelium, Normal lung",Vessels
104819,2_10,Vessels,Control,normal,BRAC3495.3f_ROI1_Cont_crop1.,20190917_BRAC3495.3f_ROI1_Cont_crop1,19_Endothelium,Vessels
104820,2_100,Vessels,Control,normal,BRAC3495.3f_ROI1_Cont_crop1.,20190917_BRAC3495.3f_ROI1_Cont_crop1,"14_Endothelium, Normal lung",Vessels
104821,2_1000,Vessels,Control,normal,BRAC3495.3f_ROI1_Cont_crop1.,20190917_BRAC3495.3f_ROI1_Cont_crop1,"14_Endothelium, Normal lung",Vessels
104822,2_10000,Myeloid:_dendritic_cells,Control,tumour,BRAC3495.3f_ROI1_Cont_crop1.,20190917_BRAC3495.3f_ROI1_Cont_crop1,27_Dendritic cells,Myeloid:_dendritic_cells
...,...,...,...,...,...,...,...,...
273652,8_9995,Myeloid:_macrophages_type_2,Control,tumour,BRAC3438.6f_ROI3_Cont.,20191121_BRAC3438.6f_ROI3_Cont,"26_Macrophages, CD206_type_2",Myeloid:_macrophages_type_2
273653,8_9996,Tumour,Control,tumour,BRAC3438.6f_ROI3_Cont.,20191121_BRAC3438.6f_ROI3_Cont,22_Tumour,Tumour
273654,8_9997,Tumour,Control,tumour,BRAC3438.6f_ROI3_Cont.,20191121_BRAC3438.6f_ROI3_Cont,22_Tumour,Tumour
273655,8_9998,Myeloid:_macrophages_type_2,Control,tumour,BRAC3438.6f_ROI3_Cont.,20191121_BRAC3438.6f_ROI3_Cont,"26_Macrophages, CD206_type_2",Myeloid:_macrophages_type_2


In [12]:
# For example, here we encode the 'Domain' column
le_domain = preprocessing.LabelEncoder()
df_control['Domain'] = le_domain.fit_transform(df_control['Domain'])

ipykernel_launcher:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [81]:
list(le_domain.inverse_transform([0,1,2]))

['interface', 'normal', 'tumour']

In [11]:
loc_all_cell_control = df_control[['Location_Center_X','Location_Center_Y']].to_numpy()
loc_all_cell_mrtx    = df_mrtx[['Location_Center_X','Location_Center_Y']].to_numpy()

loc_all_cell_control.shape, loc_all_cell_mrtx.shape

((131861, 2), (144898, 2))

In [13]:
# Create separate dataframe with the columns that will be used as features in the GCN/other algorithms
# Eg, Specify which columns will be excluded from this
cols_to_drop=['X.1', 'X.2','X.3'] # fill in which columns to drop

In [14]:
ddf_control=df_control.drop(cols_to_drop, axis=1)
ddf_mrtx=df_mrtx.drop(cols_to_drop, axis=1)

In [15]:
#control rois 
control_rois=ddf_control.ROI_name.unique()
#treated rois
mrtx_rois=ddf_mrtx.ROI_name.unique()

In [18]:
control_rois,mrtx_rois

(array(['BRAC3495.3f_ROI1_Cont_crop1.', 'BRAC3495.3f_ROI1_Cont_crop2.',
        'BRAC3326.4e_ROI1_Cont_crop1.', 'BRAC3438.6f_ROI1_Cont.',
        'BRAC3438.6f_ROI2_Cont.', 'BRAC3438.6f_ROI3_Cont.'], dtype=object),
 array(['BRAC3529.2d_ROI1_MRTX.', 'BRAC4002.3c_ROI2_MRTX_crop1.',
        'BRAC4002.3c_ROI2_MRTX_crop2.', 'BRAC4002.3c_ROI3_MRTX.',
        'BRAC3529.2b_ROI1_MRTX_crop2.', 'BRAC4002.3c_ROI1_MRTX.'],
       dtype=object))

In [19]:
#control rois 
control_rois=ddf_control.ROI_name.unique()
#treated rois
mrtx_rois=ddf_mrtx.ROI_name.unique()

# data frame for each roi. can write function to do this, or create a dictionary   
mrtx_roi_2_all = ddf_mrtx[(ddf_mrtx.ROI_name==(mrtx_rois[0]))]
mrtx_roi_1_all = ddf_mrtx[(ddf_mrtx.ROI_name==(mrtx_rois[1]))]
#
control_roi_7_all  = ddf_control[(ddf_control.ROI_name==(control_rois[2]))]
control_roi_12_all = ddf_control[(ddf_control.ROI_name==(control_rois[1]))]
mrtx_rois[1]

'BRAC4002.3c_ROI2_MRTX_crop1.'

In [88]:
# locations of all cells in each roi
loc_mrtx_roi_1_all = mrtx_roi_1_all[['Location_Center_X','Location_Center_Y']].to_numpy()
loc_mrtx_roi_2_all = mrtx_roi_2_all[['Location_Center_X','Location_Center_Y']].to_numpy()
# etc...same for control

In [89]:
# Create distance matrix. This calculates the distances between every pair of nodes within a radius of r = cutoff of every node

def distance_matrix(cutoff, points_1, points_2=None, output='dict'):
    
    """
    This function computes a distance matrix between points in 'points_1' and 'points_2'
    that are within 'cutoff' of each other.
    
    ***Input
    cutoff   : the distance cutoff
    points_1 : a numpy array of coordinates
    points_2 : a numpy array of coordinates
    output   : form of output.
    ***Output
            'dict': (default) is a dictionary with every pair of points as the keys
                                 and the distance as the value.
            'coo_matrix', 'dok_matrix', 'ndarray'
    """
    
    tree1 = scipy.spatial.cKDTree(points_1, leafsize=16)
    if points_2 is None:
        points_2 = points_1
    tree2 = scipy.spatial.cKDTree(points_2,leafsize=16)
    
    distances = tree1.sparse_distance_matrix(tree2, cutoff, output_type=output)
    return distances

In [90]:
# Calculate distance matrix for all cells in roi 1


In [91]:
# %%time
dist_1=distance_matrix(25, loc_mrtx_roi_1_all,loc_mrtx_roi_1_all, 'dict')


array([[ 854.3813,  114.592 ],
       [1071.992 ,  291.3498],
       [ 313.8542, 1066.306 ],
       ...,
       [ 158.1611, 1063.333 ],
       [ 274.    , 1066.456 ],
       [ 284.3333, 1064.59  ]])

In [92]:
# create a weighted adjacency matrix from the above distance matrix. edge weight should be inverse of distance (can also try 1/(distance^2), for a faster fall off of interactions)
# put this into whatever data structure your GCN code takes as input.

n,_  = loc_mrtx_roi_1_all.shape 
adj = np.zeros((n,n))

for e, d in dist_1.items(): 
    v1,v2 = e 
    if v1 != v2:
        adj[v1][v2] =  1/d 
        adj[v2][v1] =  1/d 


mrtx_roi_1_all = ddf_mrtx[(ddf_mrtx.ROI_name==(mrtx_rois[1]))]
mrtx_roi_2_all = ddf_mrtx[(ddf_mrtx.ROI_name==(mrtx_rois[0]))]



In [93]:
mrtx_roi_1_all.columns
cols_to_drop = ['Location_Center_X','Location_Center_Y']
mrtx_roi_1_all=mrtx_roi_1_all.drop(cols_to_drop,axis=1)


In [98]:
labels = mrtx_roi_1_all['Domain'].to_numpy()
features = mrtx_roi_1_all.drop('Domain',axis=1)
print(features.columns)
# indexes_to_encode = [2, 3, 21, 22, 23, 24, 55]



to_encode = ['cellID', 'cellType', 'Treatment', 'ROI_name', 'Filename', 'Clustername', 'clustername_tumour']

for col in to_encode: 
    le_domain = preprocessing.LabelEncoder()
    features[col] = le_domain.fit_transform(features[col])
features=features.to_numpy()



Index(['Unnamed: 0', 'X', 'cellID', 'cellType', 'B_cells', 'Epithelium',
       'Fibroblasts', 'Myeloid_dendritic_cells',
       'Myeloid_dendritic_cells_CD103', 'Myeloid_macrophages',
       'Myeloid_macrophages_type1', 'Myeloid_macrophages_type2',
       'Myeloid_neutrophils', 'Normal', 'Tumour', 'Vessels', 'T_cells_CD8',
       'T_cells_CD4', 'T_reg_cells', 'NK_cells', 'cluster', 'Treatment',
       'ROI_name', 'Filename', 'Clustername', 'ObjectNumber', 'ImageNumber',
       'MI_B220', 'MI_CD103', 'MI_CD11c', 'MI_CD206', 'MI_CD3', 'MI_CD44',
       'MI_CD45', 'MI_CD4', 'MI_CD68', 'MI_CD80', 'MI_CD86', 'MI_CD8',
       'MI_EPCAM', 'MI_F480', 'MI_Foxp3', 'MI_Ki67', 'MI_LY6G', 'MI_MHCcII',
       'MI_NKp46', 'MI_PD1', 'MI_PDL1', 'MI_PECAM', 'MI_PVR', 'MI_TCRgd',
       'MI_Vimentin', 'MI_aSMA', 'MI_casp3', 'MI_pS6', 'clustername_tumour',
       'clustername_tumourID'],
      dtype='object')


In [97]:
st=[]
l=['Unnamed: 0', 'X', 'cellID', 'cellType', 'B_cells', 'Epithelium',
       'Fibroblasts', 'Myeloid_dendritic_cells',
       'Myeloid_dendritic_cells_CD103', 'Myeloid_macrophages',
       'Myeloid_macrophages_type1', 'Myeloid_macrophages_type2',
       'Myeloid_neutrophils', 'Normal', 'Tumour', 'Vessels', 'T_cells_CD8',
       'T_cells_CD4', 'T_reg_cells', 'NK_cells', 'cluster', 'Treatment',
       'ROI_name', 'Filename', 'Clustername', 'ObjectNumber', 'ImageNumber',
       'MI_B220', 'MI_CD103', 'MI_CD11c', 'MI_CD206', 'MI_CD3', 'MI_CD44',
       'MI_CD45', 'MI_CD4', 'MI_CD68', 'MI_CD80', 'MI_CD86', 'MI_CD8',
       'MI_EPCAM', 'MI_F480', 'MI_Foxp3', 'MI_Ki67', 'MI_LY6G', 'MI_MHCcII',
       'MI_NKp46', 'MI_PD1', 'MI_PDL1', 'MI_PECAM', 'MI_PVR', 'MI_TCRgd',
       'MI_Vimentin', 'MI_aSMA', 'MI_casp3', 'MI_pS6', 'clustername_tumour',
       'clustername_tumourID']
for i,x in enumerate(features[0]): 
    if type(x)==str:
        st.append(l[i])
print(st)

['cellID', 'cellType', 'Treatment', 'ROI_name', 'Filename', 'Clustername', 'clustername_tumour']


In [100]:
features

array([1.278730e+05, 1.278730e+05, 1.000000e+00, 1.300000e+01,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 1.428571e-01,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 4.761905e-02,
       0.000000e+00, 0.000000e+00, 8.095238e-01, 0.000000e+00,
       0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       2.900000e+01, 0.000000e+00, 0.000000e+00, 0.000000e+00,
       6.000000e+00, 1.000000e+03, 3.000000e+00, 3.200013e-01,
       2.632136e-01, 4.122708e-01, 8.854261e-02, 3.900941e-02,
       7.452304e-01, 3.153110e-01, 8.647221e-02, 3.068781e-01,
       4.733572e-01, 3.892209e-01, 1.717837e-01, 5.758085e-02,
       5.949226e-02, 2.095955e-01, 3.946183e-01, 3.694697e-02,
       4.903612e-01, 3.695566e-01, 3.807219e-01, 2.455287e-01,
       4.085591e-01, 4.738021e-01, 1.231498e-01, 4.464051e-01,
       2.161955e-01, 9.827914e-02, 6.957322e-01, 1.300000e+01,
       1.500000e+01])